In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
print(tf.__version__)

1.12.0


Reading the train files from the train datset

In [3]:
labels  = pd.read_csv('labels.csv')

In [4]:
labels['breed'].nunique()

120

In [5]:
labels['breed'].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
cairn                             106
leonberg                          106
japanese_spaniel                  105
beagle                            105
australian_terrier                102
blenheim_spaniel                  102
miniature_pinscher                102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
norwegian_elkhound                 95
whippet                            95
siberian_husky                     95
pug         

Reading the data 

In [6]:
# img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
# img = image.img_to_array(img)
# tar_size = 224

In [7]:
import sys
from PIL import Image
sys.modules['Image'] = Image 


In [8]:
from PIL import Image
print(Image.__file__)

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\PIL\Image.py


In [9]:
import Image
print(Image.__file__)

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\PIL\Image.py


In [10]:
import cv2

In [11]:
img = cv2.imread('D:\\ML\\Dog_Breed\\train')

In [12]:
print(img)

None


In [13]:
import cv2
import os
im_size = 90
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            #images.append(img)
            images.append(cv2.resize(img, (im_size, im_size)))
    return images

In [14]:
images = load_images_from_folder('D:\\ML\\Dog_Breed\\train')

In [15]:
len(images)

10222

In [16]:
#running all the images using resnet using pretrained weights of imagenet

test train split

In [17]:
X = images
y = labels

In [18]:
import numpy as np

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [20]:
y_train.nunique()

id       8177
breed     120
dtype: int64

In [21]:
y_valid.nunique()

id       2045
breed     120
dtype: int64

In [22]:
from keras.applications.resnet50 import ResNet50

In [23]:
im_size = 224
base_model = ResNet50(weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [24]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 120 classes
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [25]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [33]:
print(X_train[0].shape)

(90, 90, 3)


In [28]:
#X_train = np.expand_dims(X_train, axis =0)
#X_valid = np.expand_dims(X_valid, axis =0)

model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid), verbose=1)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 8177 arrays: [array([[[254, 223, 194],
        [253, 221, 192],
        [253, 222, 192],
        ...,
        [225, 188, 166],
        [224, 189, 172],
        [202, 167, 142]],

       [[253, 221, 192],
        [...